In [18]:
import sys
import torch

device = torch.device("mps")
print(device)

sys.path.append("/Users/osinachinwogu/Documents/cybertruck")

mps


In [27]:
from cnn.face_cnn import Face_CNN
import torch.onnx


face_model = Face_CNN(None).to(device)
x = torch.randn(5, 3, 299, 299).to(device)
face_model.load_state_dict(
    torch.load(r"/Users/osinachinwogu/Documents/cybertruck/epoch10_11-28_10_50_06_66acc.pt", map_location = device),
    )

/Users/osinachinwogu/miniforge3/envs/projenv/lib/python3.9/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(


<All keys matched successfully>

In [30]:
output = face_model(x)
output.shape

torch.Size([5, 10])

In [33]:
torch.onnx.export(face_model, 
                  x, 
                  'face_cnn.onnx',
                  export_params=True, 
                  opset_version=17, 
                  do_constant_folding=True, 
                  input_names = ['input'], 
                  output_names = ['output'], 
                  dynamic_axes={'input' : {0 : 'batch_size'}, 'output' : {0 : 'batch_size'}})

In [41]:
import onnx
import onnx_tf

/Users/osinachinwogu/miniforge3/envs/projenv/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [42]:
onnx_model = onnx.load('face_cnn.onnx')
tf_model = onnx_tf.backend.prepare(onnx_model)
tf_model.export_graph("face_cnn.tf")

INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to transpose_230_x, add_43_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: face_cnn.tf/assets


INFO:tensorflow:Assets written to: face_cnn.tf/assets
INFO:absl:Writing fingerprint to face_cnn.tf/fingerprint.pb


In [43]:
import tensorflow as tf

In [44]:
tf_converter = tf.lite.TFLiteConverter.from_saved_model("face_cnn.tf")
tflite_model = tf_converter.convert()
open('face_cnn.tflite', 'wb').write(tflite_model)

2023-11-29 18:46:34.444576: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-29 18:46:34.444780: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-29 18:46:34.445536: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: face_cnn.tf
2023-11-29 18:46:34.461692: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-29 18:46:34.461713: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: face_cnn.tf
2023-11-29 18:46:34.478957: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2023-11-29 18:46:34.481957: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-29 18:46:34.567298: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: face_cnn.tf
2023-11-29 18:46:34.63

83265428

In [45]:
import numpy as np

In [50]:
import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="face_cnn.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[-0.04900261  0.25725016 -0.2126121  -0.3702479  -0.41028324 -1.0210481
  -0.03697115  0.8536001   1.3789959  -0.49312112]]


In [52]:
# Load the TFLite model in TFLite Interpreter
interpreter = tf.lite.Interpreter(model_path="face_cnn.tflite")

# Resize input shape for dynamic shape model and allocate tensor
interpreter.resize_tensor_input(interpreter.get_input_details()[0]['index'], [1, 3, 299, 299])
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details) # Resized
print(output_details)

[{'name': 'serving_default_input:0', 'index': 0, 'shape': array([  1,   3, 299, 299], dtype=int32), 'shape_signature': array([ -1,   3, 299, 299], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'PartitionedCall:0', 'index': 360, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([-1, 10], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
